In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('/content/sample_data/01_train.csv')

from sklearn.model_selection import train_test_split

train_set, test_set_from_train = train_test_split(df, test_size=0.2, random_state=42)

# split target and features
X_train = train_set.drop('Premium Amount', axis=1)
y_train = train_set['Premium Amount'].copy()

In [6]:
import tensorflow as tf
import tensorflow.keras.backend as K

def root_mean_squared_log_error(y_true, y_pred):
    msle = tf.keras.losses.MeanSquaredLogarithmicError()
    return K.sqrt(msle(y_true, y_pred))

In [7]:
X_train = tf.convert_to_tensor(X_train)
y_train = tf.convert_to_tensor(y_train)

tf.random.set_seed(42)
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(1)
])
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=root_mean_squared_log_error, optimizer=optimizer)
norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=10)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
30000/30000 ━━━━━━━━━━━━━━━━━━━━ 53s 2ms/step - loss: 1.1776
Epoch 2/10
30000/30000 ━━━━━━━━━━━━━━━━━━━━ 55s 2ms/step - loss: 1.0747
Epoch 3/10
30000/30000 ━━━━━━━━━━━━━━━━━━━━ 77s 2ms/step - loss: 1.0741
Epoch 4/10
30000/30000 ━━━━━━━━━━━━━━━━━━━━ 50s 2ms/step - loss: 1.0738
Epoch 5/10
30000/30000 ━━━━━━━━━━━━━━━━━━━━ 50s 2ms/step - loss: 1.0735
Epoch 6/10
30000/30000 ━━━━━━━━━━━━━━━━━━━━ 83s 2ms/step - loss: 1.0732
Epoch 7/10
30000/30000 ━━━━━━━━━━━━━━━━━━━━ 82s 2ms/step - loss: 1.0691
Epoch 8/10
30000/30000 ━━━━━━━━━━━━━━━━━━━━ 49s 2ms/step - loss: 1.0592
Epoch 9/10
30000/30000 ━━━━━━━━━━━━━━━━━━━━ 82s 2ms/step - loss: 1.0564
Epoch 10/10
30000/30000 ━━━━━━━━━━━━━━━━━━━━ 82s 2ms/step - loss: 1.0551


In [8]:
X_test = test_set_from_train.drop('Premium Amount', axis=1)
y_test = test_set_from_train['Premium Amount'].copy()
X_test = tf.convert_to_tensor(X_test)
y_test = tf.convert_to_tensor(y_test)

In [9]:
rmsle_test = model.evaluate(X_test, y_test)

7500/7500 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - loss: 1.0580


In [26]:
# save final prediction
test_set = pd.read_csv('/content/sample_data/01_test.csv')
del test_set['Premium Amount']
test_set = tf.convert_to_tensor(test_set)
y_pred = model.predict(test_set)

25000/25000 ━━━━━━━━━━━━━━━━━━━━ 35s 1ms/step


In [31]:
original_test_set = pd.read_csv('/content/sample_data/test.csv')
id = original_test_set['id']
y_pred = y_pred.flatten()
final_prediction = pd.DataFrame({'id': id, 'Premium Amount': y_pred})
final_prediction.to_csv('keras_sequential_01.csv', index=False)